In [1]:
import pandas as pd
import numpy as np

# these only work on my laptop - but a relative path is just as silly, and I'm not copying these files here

ccm_path = 'C:\\Users\\DoLe\\Desktop\\licORbuy-analysis\\fy_tests-03v0-04v0\\temp\\fy_ccm_plus_grantsAndTrans.dta'
edgar_path = 'C:\\Users\\DonsLaptop\\Dropbox\\Research Dropbox\\Projects\\Patents\\02 Get and Parse 10ks\\10 python to download 10ks\\list_10Ks.csv'

# Get manageable CCM

Requirements:
1. 2008 (outcome year) and 2007 (lagged controls)
2. SIC2 = 73 ("Business services, contains Apple and others)
3. Have recent patents (`stock15dep_pats_granted > 0`)
4. The firm exists (with the above filters) in both years.

Students only need to pull 169 10-Ks.

In [2]:
ccm_with_tech = pd.read_stata(ccm_path)

In [3]:
subsample = (ccm_with_tech
             .assign(sic2 = ccm_with_tech['sic3']//10)
             .query('sic2==73 & (fyear == 2007 | fyear == 2008 )')
             .query('stock15dep_pats_granted > 0')
            )


In [4]:
subsample['fyear'].value_counts() # 2007 firms need to get pulled 

2008.0    213
2007.0    213
Name: fyear, dtype: int64

In [5]:
for col in subsample.columns:
    print(col)


gvkey
lpermno
datadate
fyear
sic
sic3
td
long_debt_dum
me
l_a
l_sale
div_d
age
atr
smalltaxlosscarry
largetaxlosscarry
gdpdef
l_reala
l_reallongdebt
kz_index
ww_index
hp_index
ww_unconstrain
ww_constrained
kz_unconstrain
kz_constrained
hp_unconstrain
hp_constrained
tnic3hhi
tnic3tsimm
prodmktfluid
delaycon
equitydelaycon
debtdelaycon
privdelaycon
at_raw
raw_Inv
raw_Ch_Cash
raw_Div
raw_Ch_Debt
raw_Ch_Eqty
raw_Ch_WC
raw_CF
l_emp
l_ppent
l_laborratio
Inv
Ch_Cash
Div
Ch_Debt
Ch_Eqty
Ch_WC
CF
td_a
td_mv
mb
prof_a
ppe_a
cash_a
xrd_a
dltt_a
invopps_FG09
sales_g
dv_a
short_debt
fy_flow_or_n_license
fy_flow_or_citew_license
fy_flow_or_n_sale
fy_flow_or_citew_sale
fy_flow_ee_n_license
fy_flow_ee_citew_license
fy_flow_ee_n_sale
fy_flow_ee_citew_sale
fy_stock15_or_n_license
fy_stock15_or_citew_license
fy_stock15_or_n_sale
fy_stock15_or_citew_sale
fy_stock15_ee_n_license
fy_stock15_ee_citew_license
fy_stock15_ee_n_sale
fy_stock15_ee_citew_sale
flow_citesgiven_all
flow_citesReceived_all
flow_pats_gr

In [6]:
# keep some vars, including new (to students) technology vars
keep_vars = [
    'gvkey', 'fyear', 'datadate', 'lpermno', 'gsector', 'sic', 'sic3', 
    'age', 'tic', 'state', 'at', 'me', 'l_a', 'l_sale', 'prof_a', 'mb', 
    'ppe_a', 'capx_a', 'xrd_a', 'cash_a', 'div_d', 'td', 'td_a', 'td_mv', 
    'dltt_a', 'dv_a', 'invopps_FG09', 'sales_g', 'tnic3hhi', 'tnic3tsimm', 
    'prodmktfluid', 'delaycon', 'l_emp', 'l_ppent', 'l_laborratio',
    
    'l_stock_grant_citew', 'l_stock_trans_in_citew','l_stock_trans_out_citew'
]
subsample = subsample.filter(keep_vars)

In [7]:
subsample.rename(columns={
    'l_stock_grant_citew'     :  'Log(Own Patent Stock)', 
    'l_stock_trans_in_citew'  :  'Log(Acquired Patent Stock)',
    'l_stock_trans_out_citew' :  'Log(Divested Patent Stock)'})

,gvkey,fyear,datadate,lpermno,sic,sic3,age,me,l_a,l_sale,...,tnic3hhi,tnic3tsimm,prodmktfluid,delaycon,l_emp,l_ppent,l_laborratio,Log(Own Patent Stock),Log(Acquired Patent Stock),Log(Divested Patent Stock)
3384,1410.0,2007.0,2007-10-31,47730.0,7340.0,734.0,32.0,1176.446899,7.021685,7.950788,...,1.000000,1.0000,2.773151,NaN,4.356709,3.687504,0.552400,3.335711,0.042920,2.097397
3385,1410.0,2008.0,2008-10-31,47730.0,7340.0,734.0,33.0,832.225769,7.345954,8.195221,...,1.000000,1.0000,2.911876,-0.165274,4.615120,4.128214,0.650885,3.584452,0.036598,1.956313
4963,1618.0,2008.0,2009-04-30,38746.0,7389.0,738.0,33.0,111.465637,5.427818,4.982928,...,1.000000,1.0000,5.365895,0.065357,1.054312,3.588120,2.934141,1.867269,2.350399,0.000000
5138,1634.0,2007.0,2007-12-31,14219.0,7371.0,737.0,30.0,38.352161,4.445835,5.885187,...,0.452594,1.0874,3.344160,0.094398,1.244443,1.311301,0.552400,2.767122,3.462626,3.899152
5139,1634.0,2008.0,2008-12-31,14219.0,7371.0,737.0,31.0,14.449540,3.993677,5.649689,...,0.223225,1.1147,2.739789,0.016457,0.958200,1.406342,0.650885,2.615632,3.305624,3.740202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171772,230796.0,2007.0,2007-09-30,89678.0,7370.0,737.0,7.0,235.982285,5.238843,4.086867,...,0.214744,1.4695,6.401183,-0.008781,0.279146,1.661461,2.584115,0.615186,5.807155,4.594816
171773,230796.0,2008.0,2008-09-30,89678.0,7370.0,737.0,8.0,103.476357,4.767298,4.077690,...,0.186774,1.3466,5.955298,0.016827,0.231112,1.355062,2.403822,0.543777,5.648694,4.434078
172034,264386.0,2008.0,2008-12-31,90220.0,7372.0,737.0,4.0,822.572815,6.489749,5.743433,...,0.185237,1.5452,4.700381,0.166802,0.735728,3.494991,3.380751,2.729853,3.640196,0.367213
172089,264506.0,2007.0,2007-12-31,90290.0,7372.0,737.0,3.0,928.442261,5.723157,4.899994,...,0.065298,2.6300,9.938770,0.014440,0.411447,2.831270,3.445831,2.004516,0.635206,0.000000


In [8]:
# keep only firms in both years
subsample = subsample.loc[subsample.duplicated(subset='gvkey',keep=False)].sort_values(['gvkey','fyear'])

# check that all firms are in 2007+2008 = 4015
assert subsample.groupby('gvkey')['fyear'].sum().mean() == 4015 

subsample['gvkey'].nunique()

169

# Bring in Data for Filings

In [9]:
edgar = pd.read_csv(edgar_path).filter(['gvkey','CIK','FDATE','Form','CoName','FName'])

# convert date var to date format
edgar['FDATE'] = pd.to_datetime(edgar['FDATE'].astype(str), format='%Y%m%d') 
edgar['fyear']   = edgar['FDATE'].dt.year
edgar['fyear']   = np.where(edgar['FDATE'].dt.month < 7, edgar['fyear']-1, edgar['fyear']) 

# keep 2007 and drop duplicates (keep the last of the year, often a revision)
edgar = edgar.query('fyear == 2007')
edgar = edgar.loc[~edgar.duplicated(subset = 'gvkey',keep='last')]

In [10]:
# output
(pd
     .merge(subsample,edgar,on=['gvkey','fyear'],how='left')
     .sort_values(['gvkey','fyear'])
     .to_stata('../2007_inv_and_tech.dta')
)